In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Create DataFrame from dictionary

In [2]:
# import necessary libraries
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=a00e8ef68b50d62652296d1e277e2195787a7f7e7d7264f4660250eb52c51a0e
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [3]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lit, col, explode, row_number, concat 
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("dataFrame_from_dictionary").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/23 09:44:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Create a sample dictionary

In [5]:
my_dict = { 
    'a' : ['Honda','Volkswagen'],
    'b' : ['Audi','BMW'],
    'c' : ['Tata','Jaguar']
}

In [6]:
my_dict

{'a': ['Honda', 'Volkswagen'], 'b': ['Audi', 'BMW'], 'c': ['Tata', 'Jaguar']}

In [7]:
new_dict = {
    'Name': ['Aayush','Purbi','Avinash','Neelu'],
    'Age': [10,7,14,14],
    'Height' : [6.2,5.2,5.10,5.2]
}

In [8]:
new_dict

{'Name': ['Aayush', 'Purbi', 'Avinash', 'Neelu'],
 'Age': [10, 7, 14, 14],
 'Height': [6.2, 5.2, 5.1, 5.2]}

In [9]:
df = spark.createDataFrame(my_dict.items(),"CarBrandId STRING, Brand ARRAY <STRING>")

In [10]:
from pyspark.sql.functions import *

# Explode each value 

In [11]:
df = df.withColumn("Brand",explode("Brand"))

In [12]:
df.show()

+----------+----------+
|CarBrandId|     Brand|
+----------+----------+
|         a|     Honda|
|         a|Volkswagen|
|         b|      Audi|
|         b|       BMW|
|         c|      Tata|
|         c|    Jaguar|
+----------+----------+



In [13]:
# Create Unique Identifier 
id_spec = Window.orderBy(col('CarBrandId'))

In [14]:
df = df.withColumn("CarBrandId",concat(lit("s"),row_number().over(id_spec)))

In [15]:
df.show()

23/05/23 09:44:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/23 09:44:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/05/23 09:44:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+----------+----------+
|CarBrandId|     Brand|
+----------+----------+
|        s1|     Honda|
|        s2|Volkswagen|
|        s3|      Audi|
|        s4|       BMW|
|        s5|      Tata|
|        s6|    Jaguar|
+----------+----------+

